In [495]:
# 量化策略的流程
# 1 导入数据 （包括加载数据和更新数据）
# 2 数据标准化和分析 （数据处理，数据分析）
# 3 交易信号（什么时候买，什么时候卖）
# 4 头寸管理 （什么时候多头（加仓），空头（减仓），以及平仓，仓位管理能很好地止损止盈，降低风险）
# 5 订单管理 （包括下单，撤单，记录订单）
# 6 回测 （回测指标）
# 7 小批量实盘  （实践）
# 8 大额实盘  （实践）

# 当前建立一个简单的框架 当前仅仅支持对一个现货进行交易
# 1 导入数据 
# 2 数据合成15min bar 和 dollar bar，采用2个做比较
# 3 交易信号，先采用简单双均线
# 4 头寸管理， 当前采用全仓
# 5 订单管理
# 6 回测

In [496]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px


# 简单框架实现，不使用主流的backtrader，vectorbt之类的框架
class SimpleFrame():
    df = pd.DataFrame()
    orders = pd.DataFrame()

    def __init__(self, data: pd.DataFrame):
        self.df = data

    def resample_data(self, rule='15min'):
        data = self.df
        res_data = data.set_index('datetime')
        res_df = pd.DataFrame()
        res_df['open'] = res_data['open'].resample(rule).first()
        res_df['high'] = res_data['high'].resample(rule).max()
        res_df['low'] = res_data['low'].resample(rule).min()
        res_df['close'] = res_data['close'].resample(rule).last()
        res_df['volume'] = res_data['volume'].resample(rule).sum()
        res_df['amount'] = res_data['amount'].resample(rule).sum()
        res_df['buy_amount'] = res_data['buy_amount'].resample(rule).sum()
        res_df['amount'] = res_data['amount'].resample(rule).sum()
        self.df = res_df
        return self

    def dollar_bar(self):
        data = self.df
        m = data.amount.sum() / (len(data)) * 15
        merge_data = []
        start_index = 0
        current_amount = 0
        for i in range(len(data)):
            current_amount += data.iloc[i]['amount']
            if current_amount >= m:
                bar_data = {
                    'datetime': data.iloc[start_index]['datetime'],
                    'open': data.iloc[start_index]['open'],
                    'close': data.iloc[i]['close'],
                    'high': data.iloc[start_index:i + 1]['high'].max(),
                    'low': data.iloc[start_index:i + 1]['low'].min(),
                    'volume': data.iloc[start_index:i + 1]['amount'].sum(),
                    'buy_amount': data.iloc[start_index:i + 1]['buy_amount'].sum(),
                    'amount': current_amount,
                    'count': i - start_index + 1
                }
                merge_data.append(bar_data)
                start_index = i + 1
                current_amount = 0
        #
        self.df = pd.DataFrame(merge_data)
        return self

    def signal_ma(self, N1=5, N2=20):
        entry_list = {}
        exit_list = {}
        res_df = pd.DataFrame()
        res_df['o'] = self.df['open']
        res_df['h'] = self.df['high']
        res_df['l'] = self.df['low']
        res_df['c'] = self.df['close']
        res_df['v'] = self.df['volume']
        res_df['date'] = self.df.index
        res_df['ma_s'] = res_df['c'].rolling(N1, min_periods=1).mean()
        res_df['ma_l'] = res_df['c'].rolling(N2, min_periods=1).mean()
        last_ma_s = 0
        last_ma_l = 0
        # 默认等交易数据N2天以后再交易
        for i, (index, row) in enumerate(res_df.iterrows()):
            entry_list[index] = False
            exit_list[index] = False
            if i == N1:
                last_ma_s = row['ma_s']
            if i == N2:
                last_ma_l = row['ma_l']
            if i > N2:
                if (row['ma_s'] >= row['ma_l']) and (last_ma_s < last_ma_l):
                    entry_list[index] = True
                if (row['ma_s'] <= row['ma_l']) and (last_ma_s > last_ma_l):
                    exit_list[index] = True
                last_ma_s = row['ma_s']
                last_ma_l = row['ma_l']
        #
        res_df['entry'] = entry_list.values()
        res_df['exit'] = exit_list.values()
        self.df = res_df
        return self

    def create_orders(self, init_cash=100, fee=0.001, slider=0):
        order_list = []
        equity_list = {}
        now_cash = init_cash
        hold_num = 0
        for i, (index, row) in enumerate(self.df.iterrows()):
            equity_list[index] = hold_num * row['c'] + now_cash
            # buy
            if hold_num == 0 and row['entry']:
                # total = price * num + price*num*fee
                trade_num = now_cash /(1+fee)/ (row['c'] + slider)
                trade_fee = (row['c'] + slider) * trade_num * 0.001
                order = {
                    'datetime': index,
                    'price': row['c'] + slider,
                    'side': 'BUY',
                    'fee': trade_fee,
                    'trade_num': trade_num,
                    'pre_pos': hold_num,
                    'pos': trade_num,
                    'pre_cash': now_cash,
                    'cash': 0,
                    'equity': row['c'] * trade_num,
                }
                hold_num = trade_num
                now_cash = 0
                equity_list[index] = hold_num * row['c'] + now_cash
                order_list.append(order)
            #sell
            if hold_num > 0 and row['exit']:
                sell_cash = hold_num * row['c']
                fee_cash = sell_cash * fee
                order = {
                    'datetime': index,
                    'price': row['c'],
                    'side': 'SELL',
                    'fee': fee_cash,
                    'trade_num': hold_num,
                    'pre_pos': hold_num,
                    'pos': 0,
                    'pre_cash': 0,
                    'cash': sell_cash - fee_cash,
                    'equity': sell_cash - fee_cash 
                }
                hold_num = 0
                now_cash = sell_cash - fee_cash
                equity_list[index] = hold_num * row['c'] + now_cash
                order_list.append(order)
        #
        self.orders = pd.DataFrame(order_list)
        self.df['equity'] = equity_list.values()
        return self
    
    def print_indicates(self):
        res_df = self.df.copy()
        pre_max = res_df["equity"].cummax()
        res_df['drawdown'] = res_df['equity'] / pre_max - 1
        indicates = {}
        trade_days = pd.Series([res_df['date'].head(1).values[0],res_df['date'].tail(1).values[0]]).diff()[1]
        indicates['Start'] = pd.to_datetime(res_df['date'].head(1).values[0])
        indicates['End'] = pd.to_datetime(res_df['date'].tail(1).values[0])
        indicates['Period'] = str(trade_days)
        indicates['Start Value'] = 100
        indicates['End Value'] = round(res_df['equity'].tail(1).values[0],4)
        indicates['Max Value'] = round(res_df['equity'].max(),4)
        indicates['Min Value'] = round(res_df['equity'].min(),4)
        indicates['Total Return [%]'] = round(res_df['equity'].tail(1).values[0] - 100,2)
        indicates['Total Fees Paid'] = round(self.orders['fee'].sum(),4)
        indicates['Max Drawdown [%]'] = round(res_df['drawdown'].min()*100,2)
        indicates['Max Drawdown Date'] = res_df['drawdown'].idxmin().strftime('%Y-%m-%d %H:%M:%S')
        indicates['Total Trades'] = len(self.orders)
        indicates['Total Buy Trades'] = len(self.orders.loc[self.orders['side'] == 'BUY'])
        indicates['Total Sell Trades'] = len(self.orders.loc[self.orders['side'] == 'SELL'])
        for k,v in indicates.items():
            print("%-30s        %s" % (k, v))
    
    def show_plotly(self):
        orders_df = self.orders.copy()
        res_df = self.df.copy()
        buy_df = orders_df.loc[orders_df['entry'] == True]
        sell_df = orders_df.loc[orders_df['exit'] == True]
        fig = go.Figure(layout=go.Layout(width=1200, height=600))
        fig.add_candlestick(
            name='kline',
            x=res_df.index,
            open=res_df['o'],
            high=res_df['h'],
            low=res_df['l'],
            close=res_df['c']
        )
        fig.data[0].increasing.fillcolor = 'green'
        fig.data[0].increasing.line.color = 'green'
        fig.data[0].decreasing.fillcolor = 'red'
        fig.data[0].decreasing.line.color = 'red'
        #添加MA_S MA_L
        fig.add_scatter(x=res_df.index, y=res_df['ma_s'], mode='lines', line=dict(color='blue', width=1), name="MA_S")
        fig.add_scatter(x=res_df.index, y=res_df['ma_l'], mode='lines', line=dict(color='orange', width=1), name="MA_L")
        # 添加交易点
        fig.add_scatter(x=buy_df.index, y=buy_df['c'], mode='markers', name='buy', marker=dict(size=10, symbol="diamond"))
        fig.add_scatter(x=sell_df.index, y=sell_df['c'], mode='markers', name='sell', marker=dict(size=10, symbol="diamond"))
        fig.update(layout_xaxis_rangeslider_visible=False)
        fig.show()



In [497]:
data = pd.read_csv('../20241020/BTCUSDT.csv', parse_dates=['datetime'])
# data = data[:]

In [498]:
SF = SimpleFrame(data)
SF = SF.resample_data('1h').signal_ma()
SF.df

,o,h,l,c,v,date,ma_s,ma_l,entry,exit
datetime,,,,,,,,,,
2023-01-01 00:00:00,16590.10,16613.30,16581.24,16600.39,4269.58976,2023-01-01 00:00:00,16600.390000,16600.390000,False,False
2023-01-01 01:00:00,16600.93,16604.37,16571.01,16582.08,4844.04845,2023-01-01 01:00:00,16591.235000,16591.235000,False,False
2023-01-01 02:00:00,16581.45,16589.36,16571.80,16577.79,3340.80330,2023-01-01 02:00:00,16586.753333,16586.753333,False,False
2023-01-01 03:00:00,16577.78,16590.06,16565.10,16570.14,4044.43359,2023-01-01 03:00:00,16582.600000,16582.600000,False,False
2023-01-01 04:00:00,16570.14,16574.97,16564.09,16568.60,2622.14355,2023-01-01 04:00:00,16579.800000,16579.800000,False,False
...,...,...,...,...,...,...,...,...,...,...
2024-10-01 12:00:00,63723.47,63879.81,63652.06,63868.94,778.75286,2024-10-01 12:00:00,63605.234000,63629.265000,False,False
2024-10-01 13:00:00,63868.93,63899.99,63674.49,63749.99,816.29206,2024-10-01 13:00:00,63648.834000,63639.714000,True,False
2024-10-01 14:00:00,63750.00,64097.03,63750.00,64033.98,778.17242,2024-10-01 14:00:00,63764.030000,63659.113500,False,False


In [499]:
SF.df.loc[SF.df['entry'] == True]

,o,h,l,c,v,date,ma_s,ma_l,entry,exit
datetime,,,,,,,,,,
2023-01-03 15:00:00,16727.58,16732.64,16707.57,16729.74,5504.97638,2023-01-03 15:00:00,16712.054,16710.1190,True,False
2023-01-04 10:00:00,16701.63,16754.52,16701.50,16736.52,9182.69030,2023-01-04 10:00:00,16690.262,16686.4080,True,False
2023-01-06 02:00:00,16814.18,16847.59,16800.50,16841.93,7720.01139,2023-01-06 02:00:00,16833.444,16827.0225,True,False
2023-01-07 02:00:00,16821.58,16864.15,16821.12,16850.45,9135.64548,2023-01-07 02:00:00,16817.714,16804.6205,True,False
2023-01-08 04:00:00,16944.05,16951.92,16937.04,16945.77,4307.78731,2023-01-08 04:00:00,16935.916,16934.6565,True,False
...,...,...,...,...,...,...,...,...,...,...
2024-09-24 16:00:00,63559.93,63948.00,63540.00,63851.05,995.90843,2024-09-24 16:00:00,63404.078,63291.7700,True,False
2024-09-26 14:00:00,63593.99,63780.00,63590.23,63721.48,627.19031,2024-09-26 14:00:00,63522.264,63443.2870,True,False
2024-09-29 07:00:00,65680.01,65982.16,65617.42,65858.00,359.32420,2024-09-29 07:00:00,65720.000,65714.5330,True,False


In [500]:
SF.df.loc[SF.df['exit'] == True]

,o,h,l,c,v,date,ma_s,ma_l,entry,exit
datetime,,,,,,,,,,
2023-01-03 08:00:00,16672.78,16706.14,16647.81,16695.59,5684.61483,2023-01-03 08:00:00,16710.628,16710.7355,False,True
2023-01-03 22:00:00,16725.53,16773.00,16631.95,16661.23,15670.06766,2023-01-03 22:00:00,16708.592,16710.1045,False,True
2023-01-05 06:00:00,16827.21,16853.71,16814.93,16826.68,4037.94925,2023-01-05 06:00:00,16850.076,16852.2435,False,True
2023-01-06 13:00:00,16823.24,16823.63,16792.12,16810.18,6230.44034,2023-01-06 13:00:00,16826.318,16833.0365,False,True
2023-01-07 20:00:00,16918.30,16920.07,16908.00,16916.14,5565.59897,2023-01-07 20:00:00,16923.044,16927.1715,False,True
...,...,...,...,...,...,...,...,...,...,...
2024-09-23 20:00:00,63435.03,63540.00,63236.98,63315.80,962.19152,2024-09-23 20:00:00,63475.100,63529.4050,False,True
2024-09-25 17:00:00,63819.09,63873.50,63600.00,63609.89,555.06060,2024-09-25 17:00:00,63849.790,63966.0765,False,True
2024-09-28 16:00:00,65824.01,65824.01,65578.22,65635.90,446.73728,2024-09-28 16:00:00,65844.042,65914.3490,False,True


In [501]:
SF = SF.create_orders(init_cash=100, fee=0.001)
SF.orders

,datetime,price,side,fee,trade_num,pre_pos,pos,pre_cash,cash,equity
0,2023-01-03 15:00:00,16729.74,BUY,0.099900,0.005971,0.000000,0.005971,100.000000,0.000000,99.900100
1,2023-01-03 22:00:00,16661.23,SELL,0.099491,0.005971,0.005971,0.000000,0.000000,99.391508,99.391508
2,2023-01-04 10:00:00,16736.52,BUY,0.099292,0.005933,0.000000,0.005933,99.391508,0.000000,99.292216
3,2023-01-05 06:00:00,16826.68,SELL,0.099827,0.005933,0.005933,0.000000,0.000000,99.727278,99.727278
4,2023-01-06 02:00:00,16841.93,BUY,0.099628,0.005915,0.000000,0.005915,99.727278,0.000000,99.627650
...,...,...,...,...,...,...,...,...,...,...
1026,2024-09-29 07:00:00,65858.00,BUY,0.110388,0.001676,0.000000,0.001676,110.498842,0.000000,110.388454
1027,2024-09-29 15:00:00,65477.01,SELL,0.109750,0.001676,0.001676,0.000000,0.000000,109.640104,109.640104
1028,2024-09-29 23:00:00,65751.34,BUY,0.109531,0.001666,0.000000,0.001666,109.640104,0.000000,109.530573
1029,2024-09-30 09:00:00,64769.99,SELL,0.107896,0.001666,0.001666,0.000000,0.000000,107.787915,107.787915


In [502]:
SF.orders['fee'].sum()

123.42868797778397

In [503]:
#
SF.df

,o,h,l,c,v,date,ma_s,ma_l,entry,exit,equity
datetime,,,,,,,,,,,
2023-01-01 00:00:00,16590.10,16613.30,16581.24,16600.39,4269.58976,2023-01-01 00:00:00,16600.390000,16600.390000,False,False,100.000000
2023-01-01 01:00:00,16600.93,16604.37,16571.01,16582.08,4844.04845,2023-01-01 01:00:00,16591.235000,16591.235000,False,False,100.000000
2023-01-01 02:00:00,16581.45,16589.36,16571.80,16577.79,3340.80330,2023-01-01 02:00:00,16586.753333,16586.753333,False,False,100.000000
2023-01-01 03:00:00,16577.78,16590.06,16565.10,16570.14,4044.43359,2023-01-01 03:00:00,16582.600000,16582.600000,False,False,100.000000
2023-01-01 04:00:00,16570.14,16574.97,16564.09,16568.60,2622.14355,2023-01-01 04:00:00,16579.800000,16579.800000,False,False,100.000000
...,...,...,...,...,...,...,...,...,...,...,...
2024-10-01 12:00:00,63723.47,63879.81,63652.06,63868.94,778.75286,2024-10-01 12:00:00,63605.234000,63629.265000,False,False,107.787915
2024-10-01 13:00:00,63868.93,63899.99,63674.49,63749.99,816.29206,2024-10-01 13:00:00,63648.834000,63639.714000,True,False,107.680235
2024-10-01 14:00:00,63750.00,64097.03,63750.00,64033.98,778.17242,2024-10-01 14:00:00,63764.030000,63659.113500,False,False,108.159923


In [504]:
# SF.print_indicates(res_df)

In [505]:
# vectorbt 回测 pip install -U vectorbt
import vectorbt as vbt

pf = vbt.Portfolio.from_signals(SF.df['c'], entries=SF.df['entry'], exits=SF.df['exit'],price=SF.df['c'],init_cash=100,fees=0.001)
pf.stats()


Start                         2023-01-01 00:00:00
End                           2024-10-01 16:00:00
Period                          639 days 17:00:00
Start Value                                 100.0
End Value                              108.031585
Total Return [%]                         8.031585
Benchmark Return [%]                   285.280105
Max Gross Exposure [%]                      100.0
Total Fees Paid                        123.428688
Max Drawdown [%]                        36.754103
Max Drawdown Duration           350 days 20:00:00
Total Trades                                  516
Total Closed Trades                           515
Total Open Trades                               1
Open Trade PnL                            0.24367
Win Rate [%]                            28.932039
Best Trade [%]                          18.606155
Worst Trade [%]                         -5.696785
Avg Winning Trade [%]                     2.29763
Avg Losing Trade [%]                    -0.878548


In [506]:
pf.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,0,2023-01-03 15:00:00,0.005971,16729.74,0.099900,Buy
1,1,0,2023-01-03 22:00:00,0.005971,16661.23,0.099491,Sell
2,2,0,2023-01-04 10:00:00,0.005933,16736.52,0.099292,Buy
3,3,0,2023-01-05 06:00:00,0.005933,16826.68,0.099827,Sell
4,4,0,2023-01-06 02:00:00,0.005915,16841.93,0.099628,Buy
...,...,...,...,...,...,...,...
1026,1026,0,2024-09-29 07:00:00,0.001676,65858.00,0.110388,Buy
1027,1027,0,2024-09-29 15:00:00,0.001676,65477.01,0.109750,Sell
1028,1028,0,2024-09-29 23:00:00,0.001666,65751.34,0.109531,Buy
1029,1029,0,2024-09-30 09:00:00,0.001666,64769.99,0.107896,Sell


In [507]:
pf.cash()

datetime
2023-01-01 00:00:00    100.000000
2023-01-01 01:00:00    100.000000
2023-01-01 02:00:00    100.000000
2023-01-01 03:00:00    100.000000
2023-01-01 04:00:00    100.000000
                          ...    
2024-10-01 12:00:00    107.787915
2024-10-01 13:00:00      0.000000
2024-10-01 14:00:00      0.000000
2024-10-01 15:00:00      0.000000
2024-10-01 16:00:00      0.000000
Freq: h, Length: 15353, dtype: float64

In [508]:
res_orders = pf.orders.records_readable
equity_list = {}
cash_list = {}
last_equity = 100
for i, (index, row) in enumerate(res_orders.iterrows()):
    if row['Side']=='Buy':
        equity_list[index] = row['Price']*row['Size']
        cash_list[index] = 0
    if row['Side']=='Sell':
        equity_list[index] = row['Price']*row['Size'] - row['Fees']
        cash_list[index] = equity_list[index]
    last_equity = equity_list[index]
    #
res_orders['Cash'] = cash_list.values()
res_orders['Equity'] = equity_list.values()
res_orders

,Order Id,Column,Timestamp,Size,Price,Fees,Side,Cash,Equity
0,0,0,2023-01-03 15:00:00,0.005971,16729.74,0.099900,Buy,0.000000,99.900100
1,1,0,2023-01-03 22:00:00,0.005971,16661.23,0.099491,Sell,99.391508,99.391508
2,2,0,2023-01-04 10:00:00,0.005933,16736.52,0.099292,Buy,0.000000,99.292216
3,3,0,2023-01-05 06:00:00,0.005933,16826.68,0.099827,Sell,99.727278,99.727278
4,4,0,2023-01-06 02:00:00,0.005915,16841.93,0.099628,Buy,0.000000,99.627650
...,...,...,...,...,...,...,...,...,...
1026,1026,0,2024-09-29 07:00:00,0.001676,65858.00,0.110388,Buy,0.000000,110.388454
1027,1027,0,2024-09-29 15:00:00,0.001676,65477.01,0.109750,Sell,109.640104,109.640104
1028,1028,0,2024-09-29 23:00:00,0.001666,65751.34,0.109531,Buy,0.000000,109.530573
1029,1029,0,2024-09-30 09:00:00,0.001666,64769.99,0.107896,Sell,107.787915,107.787915


In [509]:
SF.orders

,datetime,price,side,fee,trade_num,pre_pos,pos,pre_cash,cash,equity
0,2023-01-03 15:00:00,16729.74,BUY,0.099900,0.005971,0.000000,0.005971,100.000000,0.000000,99.900100
1,2023-01-03 22:00:00,16661.23,SELL,0.099491,0.005971,0.005971,0.000000,0.000000,99.391508,99.391508
2,2023-01-04 10:00:00,16736.52,BUY,0.099292,0.005933,0.000000,0.005933,99.391508,0.000000,99.292216
3,2023-01-05 06:00:00,16826.68,SELL,0.099827,0.005933,0.005933,0.000000,0.000000,99.727278,99.727278
4,2023-01-06 02:00:00,16841.93,BUY,0.099628,0.005915,0.000000,0.005915,99.727278,0.000000,99.627650
...,...,...,...,...,...,...,...,...,...,...
1026,2024-09-29 07:00:00,65858.00,BUY,0.110388,0.001676,0.000000,0.001676,110.498842,0.000000,110.388454
1027,2024-09-29 15:00:00,65477.01,SELL,0.109750,0.001676,0.001676,0.000000,0.000000,109.640104,109.640104
1028,2024-09-29 23:00:00,65751.34,BUY,0.109531,0.001666,0.000000,0.001666,109.640104,0.000000,109.530573
1029,2024-09-30 09:00:00,64769.99,SELL,0.107896,0.001666,0.001666,0.000000,0.000000,107.787915,107.787915


In [513]:
pf.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '36a8e4b3-d0cc-4c0e-a271-0633686f3104',
              'x': array([datetime.datetime(2023, 1, 1, 0, 0),
                          datetime.datetime(2023, 1, 1, 1, 0),
                          datetime.datetime(2023, 1, 1, 2, 0), ...,
                          datetime.datetime(2024, 10, 1, 14, 0),
                          datetime.datetime(2024, 10, 1, 15, 0),
                          datetime.datetime(2024, 10, 1, 16, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([16600.39, 16582.08, 16577.79, ..., 64033.98, 64045.92, 63958.  ]),
              'yaxis': 'y'},
             {'customdata': array([[0.00000000e+00, 5.97140780e-03, 9.99000999e-02],
                                   [2.00000000e+00, 5.93266793e-03, 9.92922155e-02],
                       

In [511]:
SF.print_indicates()

Start                                 2023-01-01 00:00:00
End                                   2024-10-01 16:00:00
Period                                639 days 16:00:00
Start Value                           100
End Value                             108.0316
Max Value                             155.8387
Min Value                             98.5616
Total Return [%]                      8.03
Total Fees Paid                       123.4287
Max Drawdown [%]                      -36.75
Max Drawdown Date                     2024-09-09 05:00:00
Total Trades                          1031
Total Buy Trades                      516
Total Sell Trades                     515


In [512]:
pf.stats()

Start                         2023-01-01 00:00:00
End                           2024-10-01 16:00:00
Period                          639 days 17:00:00
Start Value                                 100.0
End Value                              108.031585
Total Return [%]                         8.031585
Benchmark Return [%]                   285.280105
Max Gross Exposure [%]                      100.0
Total Fees Paid                        123.428688
Max Drawdown [%]                        36.754103
Max Drawdown Duration           350 days 20:00:00
Total Trades                                  516
Total Closed Trades                           515
Total Open Trades                               1
Open Trade PnL                            0.24367
Win Rate [%]                            28.932039
Best Trade [%]                          18.606155
Worst Trade [%]                         -5.696785
Avg Winning Trade [%]                     2.29763
Avg Losing Trade [%]                    -0.878548
